In [1]:
# search in the hyperparameter space with W&B sweep
import logging
from ast import literal_eval as load
import pandas as pd
import numpy as np
from ast import literal_eval
import sklearn

from simpletransformers.classification import (
    ClassificationArgs,
    ClassificationModel,
)

In [2]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

df = pd.read_csv('triples.csv')
df.insert(loc=0, column='idx', value=np.arange(len(df)))
sent_num=len(df)

In [3]:
def convert(arr):
    ls=[]
    trip_list=[]
    for i in range(len(arr)):
        lth = len(load(arr[i,4]))
        for p1 in range(lth-1):
            for p2 in range(p1+1, p1+2): #p1+2 or lth
                phrase1=load(arr[i,4])[p1]
                phrase2=load(arr[i,4])[p2]
                a_ls= load(arr[i,5])
                possible=1
                for a in a_ls:
                    if (a[0]==phrase1[0] and a[2]==phrase2[0]) or (a[0]==phrase2[0] and a[2]==phrase1[0]):
                        possible=0
                        break
                if possible==1:
                    word_ls=arr[i,2].split(' ')
                    triple=[phrase1[0],'has',phrase2[0]]
                    trip_list.append(triple)
                    word_ls.insert(phrase1[1][0], '<<')
                    word_ls.insert(phrase1[1][1]+1, '>>')
                    word_ls.insert(phrase2[1][0]+2, '[[')
                    word_ls.insert(phrase2[1][1]+3, ']]')
                    flg=0
                    trip_ls=load(arr[i,6])
                    for trip in trip_ls:
                        if trip[0]==phrase1[0] and trip[2]==phrase2[0]:
                            if trip[1]=='has':
                                flg=1
                                break
                            elif trip[1]=='name':
                                flg=2
                                break
                    ls.append([int(arr[i, 0]),phrase1[0],phrase2[0],trip_ls,' '.join(word_ls),flg])
    dataframe = pd.DataFrame(ls)
    dataframe.columns = ['idx','phrase1', 'phrase2', 'triples', 'text', 'labels']
    return dataframe,trip_list

In [4]:
df=df.values
df,trip_list=convert(df)

In [5]:
model_args = ClassificationArgs()

model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.manual_seed = 1
model_args.fp16 = False
model_args.use_multiprocessing = True
model_args.do_lower_case = True  # when using uncased model

In [6]:
def triple_F1(ref, pred):
    return 0

In [7]:
# Create a TransformerModel
model = ClassificationModel(
    "bert",
    "../rel/outputsB/best_model",
    args=model_args,
)
result, model_outputs, wrong_predictions = model.eval_model(df, F1_score=triple_F1)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


  0%|          | 0/6887 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/861 [00:00<?, ?it/s]

/opt/conda/envs/hl/lib/python3.9/site-packages/sklearn/metrics/_classification.py:870: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
INFO:simpletransformers.classification.classification_model:{'mcc': 0.0, 'F1_score': 0, 'eval_loss': 0.6911000259916659}


In [8]:
preds = list(model_outputs.argmax(axis=1))
df['preds']=preds
df['cand']=trip_list
df.loc[df['preds']==0,'cand']=None
#df.loc[df['preds']==2,'cand'][1]='name'
for i in range(len(df)):
    if df.loc[i,'preds']==2:
        df.loc[i,'cand'][1]='name'
data=[]
for i in range(sent_num):
    temp = list(df[df['idx']==i]['cand'])
    temp = [t for t in temp if t]
    data.append(str(temp))

In [9]:
data

['[]',
 '[]',
 '[]',
 "[['two variants', 'has', 'algorithm']]",
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 "[['pretraining', 'has', 'bi-directional transformer model']]",
 '[]',
 '[]',
 "[['bi-directional transformer architecture', 'has', 'predicts'], ['predicts', 'has', 'every token']]",
 "[['cloze - style training objective', 'has', 'model']]",
 '[]',
 '[]',
 "[['dimensionality', 'has', '1024'], ['dimensionality', 'has', '256'], ['momentum', 'has', '0.99']]",
 "[['learning rate', 'has', 'linearly warmed up']]",
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 "[['much better', 'name', 'MRPC']]",
 '[]',
 "[['fine tuning', 'has', 'biggest gain']]",
 '[]',
 "[['scaling', 'has', 'bilm term']]",
 '[]',
 '[]',
 "[['Seq2seq approach', 'has', 'stronger baseline']]",
 '[]',
 "[['L = 2 , H = 200 , and B = 5', 'has', 'adequate']]",
 "[['subword split', 'has', 'input token unit']]",
 "[['subword information', 'has', 'features']]",
 "[['current top - notch methods', 'name', 'RNNG']]",
 '[]',
 '[

In [10]:
data=pd.DataFrame(data,columns=['triple_B'])
data.to_csv('B.csv',index=False)